# Лабораторная работа 4.  Алгоритмы на графах

**Цель работы:** изучение некоторых алгоритмов на графах; исследование эффективности этих алгоритмов.

### Метод динамического программирования

### Задание 1

| Вариант | Начальная и конечная вершины| Граф |
|:---|:---|:---|
| 16 | 2, 7 | ![](./img/006.png) |

Начальные условия $f_2=0$, $S_{22} = 0$.

Находим последовательно значения функции $f_i$ (в условных единицах) для каждой вершины ориентированного графа:

$f_4 = \min(S_{42} + f_2) = 3 + f_2 = 3 + 0 = 3$;

$f_5 = \min(S_{52} + f_2) = 6 + f_2 = 6 + 0 = 6$;

$f_6 = \min 
\left(
  \begin{array}{c}
  S_{64} + f_4 \\
  S_{63} + f_3 \\
  \end{array}
\right) 
= 
\min 
\left(
  \begin{array}{c}
  4 + 3 \\
  7 + None \\
  \end{array}
\right) = 7$; 


$f_7 = \min 
\left(
  \begin{array}{c}
  S_{74} + f_4 \\
  S_{75} + f_5 \\
  S_{76} + f_6 \\
  \end{array}
\right)
= 
\min 
\left(
  \begin{array}{c}
  35 + 3 \\
  20 + 6 \\
  5 + 7 \\
  \end{array}
\right) = 12$; 


Длина кратчайшего пути составляет 12 условных единиц. 

### Задание 2

- Реализовать прогрмно поиск кратчайшего пути на графе между парами вершин из задания 1 методом динамического программирования.

In [19]:
def path_finding(graph):
    n = len(graph)
    dist = [[float('inf') for _ in range(n)] for _ in range(n)]

    # Инициализация матрицы расстояний dist
    for i in range(n):
        for j in range(n):
            if i == j:
                dist[i][j] = 0
            elif graph[i][j] != 0:
                dist[i][j] = graph[i][j]

    # Обновление матрицы расстояний
    for k in range(n):
        for i in range(n):
            for j in range(n):
                if dist[i][k] + dist[k][j] < dist[i][j]:
                    dist[i][j] = dist[i][k] + dist[k][j]

    return dist

In [20]:
# Решение задачи
# Граф (матрица смежности)
# Неподписанные пути я брал = 1
graph = [
    [0, 2, 13, 25, 17, 0, 0],
    [0, 0, 0, 3, 6, 0, 0],
    [0, 0, 0, 1, 0, 7, 0],
    [0, 0, 1, 0, 1, 4, 35],
    [0, 0, 0, 0, 0, 0, 20],
    [0, 0, 0, 0, 0, 0, 5],
    [0, 0, 0, 0, 0, 0, 0],
]

# Находим кратчайшие пути между всеми парами вершин
distances = path_finding(graph)


print(f'Кратчайший путь между вершинами {2} и {7}: {distances[1][6]}')
# # Выводим все результаты
# for i in range(len(distances)):
#     for j in range(len(distances)):
#         if i != 0 and j != 0:
#             if distances[i][j] == float('inf'):
#                 print(f'Между вершинами {i+1} и {j+1} пути не существует')
#             else:
#                 print(f'Кратчайший путь между вершинами {i+1} и {j+1}: {distances[i][j]}')

Кратчайший путь между вершинами 2 и 7: 12


### Задание 3:
Реализовать алгоритм Дейкстры поиска кратчайшего пути на графе между парами вершин:

| Вариант | Начальная и конечная вершины| Граф | 
|:---|:---|:---|
| 16 | 2, 7 | ![](./img/016.png) |

In [21]:
import heapq

def dijkstra(graph, start):
    visited = set()
    distances = {vertex: float('inf') for vertex in graph}
    distances[start] = 0
    predecessors = {vertex: None for vertex in graph}
    pq = [(0, start)]

    while pq:
        current_distance, current_vertex = heapq.heappop(pq)

        if current_vertex in visited:
            continue
  
        visited.add(current_vertex)

        for neighbor, weight in graph[current_vertex].items():
            distance = current_distance + weight

            if distance < distances[neighbor]:
                distances[neighbor] = distance
                predecessors[neighbor] = current_vertex
                heapq.heappush(pq, (distance, neighbor))

    return distances, predecessors

In [22]:
graph = {
    '1': {'2': 14, '6': 8},
    '2': {'1': 14, '3': 5, '4': 10, '5': 2, '6': 2, '8': 9},
    '3': {'2': 5, '8': 11},
    '4': {'2': 10, '5': 3, '6': 6, '7': 5},
    '5': {'2': 2, '4': 3, '7': 8, '8': 1},
    '6': {'1': 8, '2': 2, '4': 6, '7': 5},
    '7': {'4': 5, '5': 8, '6': 5, '8': 7},
    '8': {'3': 11, '5': 1, '7': 7}
}

distances, predecessors = dijkstra(graph, '2')

print(distances['7'])
print(predecessors)

7
{'1': '6', '2': None, '3': '2', '4': '5', '5': '2', '6': '2', '7': '6', '8': '5'}


### Задание 4. Алгоритм Йена

In [23]:
from heapq import heappop, heappush
from copy import deepcopy

def yen(graph, start, end, K):
    if start > end:
        start, end = end, start
    # находим кратчайший путь между начальной и конечной вершинами
    dist, prev = dijkstra(graph, start, end)
    paths = []
    paths.append(get_path(prev, end))
    
    # выполняем K раз
    for k in range(1, K):
        # делим путь на подпути и находим вес каждого подпути
        A = paths[-1]
        for i in range(len(A) - 1):
            edge_removed = (A[i], A[i+1])
            
            # создаем новый граф с удаленным ребром
            graph_removed = deepcopy(graph)
            graph_removed[edge_removed[0]][edge_removed[1]] = float('inf')
            
            # ищем кратчайший путь между начальной и конечной вершинами
            dist, prev = dijkstra(graph_removed, start, end)
            if prev[end] is None:
                break
            
            # добавляем новый путь в список путей
            B = get_path(prev, end)
            if B not in paths:
                paths.append(B)
        if len(paths) < k+1:
            break
    
    # возвращаем K кратчайших путей
    return paths[:K]

def dijkstra(graph, start, end):
    # инициализация расстояний и предыдущих вершин
    dist = {v: float('inf') for v in graph}
    dist[start] = 0
    prev = {v: None for v in graph}
    
    # очередь с приоритетом
    queue = [(dist[v], v) for v in graph]
    while queue:
        u_dist, u = heappop(queue)
        if u == end:
            break
        for neighbor in graph[u]:
            alt = u_dist + graph[u][neighbor]
            if alt < dist[neighbor]:
                dist[neighbor] = alt
                prev[neighbor] = u
                heappush(queue, (dist[neighbor], neighbor))
    
    # возвращаем расстояния и предыдущие вершины
    return dist, prev

def get_path(prev, v):
    # возвращает путь от начальной вершины до v
    path = []
    while v is not None:
        path.append(v)
        v = prev[v]
    return path[::-1]

# Пример использования:

graph = {
    '1': {'2': 14, '6': 8},
    '2': {'1': 14, '3': 5, '4': 10, '5': 2, '6': 2, '8': 9},
    '3': {'2': 5, '8': 11},
    '4': {'2': 10, '5': 3, '6': 6, '7': 5},
    '5': {'2': 2, '4': 3, '7': 8, '8': 1},
    '6': {'1': 8, '2': 2, '4': 6, '7': 5},
    '7': {'4': 5, '5': 8, '6': 5, '8': 7},
    '8': {'3': 11, '5': 1, '7': 7}
}

start = '2'
end = '7'
K = 3

paths = yen(graph, start, end, K)
for i, path in enumerate(paths):
    print(f'Path {i+1}: {path}, distance: {sum(graph[path[i]][path[i+1]] for i in range(len(path)-1))}')


Path 1: ['2', '6', '7'], distance: 7
Path 2: ['2', '5', '7'], distance: 10
